In [34]:
import plotly.express as px
import yfinance as yf

In [5]:
ticker_symbol = "DX-Y.NYB"
ticker = yf.Ticker(ticker_symbol)
df = ticker.history(period="5y").reset_index(drop=False).drop(columns=['Volume', 'Dividends', 'Stock Splits'])
df

,Date,Open,High,Low,Close
0,2020-11-02 00:00:00-05:00,93.889999,94.290001,93.889999,94.050003
1,2020-11-03 00:00:00-05:00,94.050003,94.050003,93.290001,93.550003
2,2020-11-04 00:00:00-05:00,93.089996,94.300003,93.089996,93.410004
3,2020-11-05 00:00:00-05:00,93.339996,93.550003,92.489998,92.529999
4,2020-11-06 00:00:00-05:00,92.570000,92.820000,92.180000,92.230003
...,...,...,...,...,...
1253,2025-10-27 00:00:00-04:00,98.839996,99.000000,98.730003,98.779999
1254,2025-10-28 00:00:00-04:00,98.750000,98.949997,98.570000,98.690002
1255,2025-10-29 00:00:00-04:00,98.739998,99.360001,98.620003,99.220001
1256,2025-10-30 00:00:00-04:00,99.139999,99.720001,98.919998,99.529999


# Question 1

In [33]:

fig = px.line(df, x="Date", y=["Close", "High", "Low", "Open"], labels={"value": "USD Price", "variable": "Date"})
fig.show()

# Question 2

The time-series are non-stationary because the price level doesn't fluctuate around a central value. It moves from ~90 in 2021, ~110 in 2022, drops to ~105 in 2023 and then fluctuates between ~100 to ~110 in 2024-2025. Also there is a trend in 2022 (upward) and in 2025 there is volatility. Variance also changes: relatively stable in 2023 but highly volatile in 2022 and 2025.

# Question 3

Training a regressor on these non-stationary time series would cause problems because the model assumes data stay relatively constant over time. Our price moves from ~90 to ~115 to ~100, with high volatility in 2022 and low volatility in 2023. A model trained on 2021-2022 upward trend would wrongly predict prices keep rising and fail completely when they drop in late 2022. The model would also treat temporary events like the spike to 115 as recurring patterns when they might be one-time occurrences. If we train on volatile 2022, the model would probably expect large swings in calm 2023 that don't happen, while training on stable 2023 would leave it unprepared for 2025 volatility. The issue is that regression cannot handle the changing mean and variance in non-stationary data.

# Question 4

We notice that the price of dollar dropped to very low, same period Trump imposed tariffs which cuased financial worry for the economy.

# Question 5

No, we can't predict tomorrow's close price using today's open, high, low, and close. Today's open, high, low and close might be irrelevant tomorrow (eg due to tariffs) because usd price is non-stationary and is affected by data not present in our dataset.